In [12]:
from fastFM import als 
import pandas as pd
from numpy.linalg import norm
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
from sklearn.metrics import mean_squared_error,recall_score,ndcg_score
from sklearn.model_selection import ShuffleSplit
from scipy.sparse import csc_matrix
import json
'''
https://github.com/ibayer/fastFM
'''

'\nhttps://github.com/ibayer/fastFM\n'

In [4]:
user_movie = np.load('../hw3_data/Movielens/user_movie.npy')
embed_X = np.load("../hw3_data/Movielens/embedding.npy")

X = embed_X 
y = user_movie[:,-1]

In [9]:
user_business = np.load('../hw3_data/Yelp/user_business.npy')
X = np.load("../hw3_data/Yelp/Yelp_embedding.npy")

X = embed_X 
y = user_business[:,-1]

In [7]:
def Average(lst): 
	return round(sum(lst) / len(lst),4) 

In [13]:
RMSE_M = []
recall_M = []
NDCG_M = []
store_final_result = {}
rs = ShuffleSplit(n_splits=5, test_size=.1, random_state=42)
for i, (train_index, test_index) in enumerate(rs.split(X)):
    X_train, y_train = X[train_index], y[train_index]
    X_test, y_test = X[test_index], y[test_index]
    X_train = csc_matrix(X_train)
    X_test = csc_matrix(X_test)
    fm = als.FMRegression(n_iter=1000, init_stdev=0.1, rank=2, l2_reg_w=0.1, l2_reg_V=0.5)
    fm.fit(X_train, y_train)
    y_pred = fm.predict(X_test)

    RMSE = mean_squared_error(np.array(y_pred),y_test, squared=False)
    RMSE_M.append(RMSE)
    recall = recall_score(np.round(np.array(y_pred)),y_test, average='micro')
    recall_M.append(recall)
    NDCG = ndcg_score(np.array([y_pred]),y_test.reshape(1, -1), k=10)
    NDCG_M.append(NDCG) 
    print(f"RMSE: {RMSE}, Recall: {recall}, NDCG: {NDCG}")
print(f"Average RMSE: {Average(RMSE_M)}, Average Recall: {Average(recall_M)}, Average NDCG: {Average(NDCG_M)}") 
store_final_result['Yelp'] = f"Average RMSE: {Average(RMSE_M)}, Average Recall: {Average(recall_M)}, Average NDCG: {Average(NDCG_M)}"

#0.8353, Average Recall: 0.4685, Average NDCG: 0.7758
with open('final_result_yelp.txt', 'w') as file:
    file.write(json.dumps(store_final_result))

In [74]:
user_index, item_index, fea_age, fea_occupation, fea_genre, user_knn, item_knn

(627, 649)